In [183]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [258]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [259]:
dataa.columns

Index(['instance_id', 'item_category_list_num', 'item_property_list_num',
       'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'property_0_pvnum', 'property_1_pvnum',
       'property_2_pvnum', 'item_id_pvnum', 'occupation_click',
       'user_gender_click'],
      dtype='object')

In [260]:
tezheng = ['instance_id','item_id_pvnum', 'occupation_click','category_1_pvnum', 'shop_click', 'brand_click', 'item_click', 'item_category_list_num', 'item_property_list_num']

In [261]:
dataa = dataa[tezheng]

In [262]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [263]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 39)
2018-09-24 23:59:47
(357066, 39)
(121021, 39)


In [264]:

print('test')
test_a = pd.read_csv('test.csv',index_col=False)
data2a = pd.read_csv('test_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [265]:
data2a = data2a[tezheng]

In [266]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [267]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
del train['context_id']
del val['context_id']
del test_a['context_id']
del train['item_brand_id']
del val['item_brand_id']
del test_a['item_brand_id']
del train['category_0']
del val['category_0']
del test_a['category_0']
del train['category_2']
del val['category_2']
del test_a['category_2']
del train['property_1']
del val['property_1']
del test_a['property_1']
del train['property_2']
del val['property_2']
del test_a['property_2']

(18371, 34)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [268]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [269]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(18371, 31)


In [270]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
del train['item_id']
del test_a['item_id']
del val['item_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_page_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'item_id_pvnum', 'occupation_click', 'category_1_pvnum', 'shop_click',
       'brand_click', 'item_click', 'item_category_list_num',
       'item_property_list_num', 'category_1', 'property_0', 'halfhour',
       'user_count', 'item_count'],
      dtype='object')
(496458, 260)


In [271]:
# LogisticRegression?

In [272]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [273]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [274]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
#                         max_depth = 8,
                        
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 1
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683894
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674839
[3]	valid_0's binary_logloss: 0.665954
[4]	valid_0's binary_logloss: 0.657241
[5]	valid_0's binary_logloss: 0.648693
[6]	valid_0's binary_logloss: 0.640311
[7]	valid_0's binary_logloss: 0.632081
[8]	valid_0's binary_logloss: 0.624011
[9]	valid_0's binary_logloss: 0.616088
[10]	valid_0's binary_logloss: 0.608313
[11]	valid_0's binary_logloss: 0.600683
[12]	valid_0's binary_logloss: 0.593192
[13]	valid_0's binary_logloss: 0.585835
[14]	valid_0's binary_logloss: 0.578613
[15]	valid_0's binary_logloss: 0.571519
[16]	valid_0's binary_logloss: 0.564552
[17]	valid_0's binary_logloss: 0.557702
[18]	valid_0's binary_logloss: 0.550972
[19]	valid_0's binary_logloss: 0.544362
[20]	valid_0's binary_logloss: 0.537865
[21]	valid_0's binary_logloss: 0.531486
[22]	valid_0's binary_logloss: 0.525202
[23]	valid_0's binary_logloss: 0.519028
[24]	valid_0's binary_logl

[208]	valid_0's binary_logloss: 0.122979
[209]	valid_0's binary_logloss: 0.122461
[210]	valid_0's binary_logloss: 0.121949
[211]	valid_0's binary_logloss: 0.121444
[212]	valid_0's binary_logloss: 0.120946
[213]	valid_0's binary_logloss: 0.120453
[214]	valid_0's binary_logloss: 0.119966
[215]	valid_0's binary_logloss: 0.119484
[216]	valid_0's binary_logloss: 0.11901
[217]	valid_0's binary_logloss: 0.118542
[218]	valid_0's binary_logloss: 0.118077
[219]	valid_0's binary_logloss: 0.117618
[220]	valid_0's binary_logloss: 0.117168
[221]	valid_0's binary_logloss: 0.116725
[222]	valid_0's binary_logloss: 0.116286
[223]	valid_0's binary_logloss: 0.115852
[224]	valid_0's binary_logloss: 0.115426
[225]	valid_0's binary_logloss: 0.115001
[226]	valid_0's binary_logloss: 0.114582
[227]	valid_0's binary_logloss: 0.114169
[228]	valid_0's binary_logloss: 0.113759
[229]	valid_0's binary_logloss: 0.113354
[230]	valid_0's binary_logloss: 0.112953
[231]	valid_0's binary_logloss: 0.112561
[232]	valid_0's b

[410]	valid_0's binary_logloss: 0.085256
[411]	valid_0's binary_logloss: 0.0852203
[412]	valid_0's binary_logloss: 0.0851818
[413]	valid_0's binary_logloss: 0.085147
[414]	valid_0's binary_logloss: 0.0851125
[415]	valid_0's binary_logloss: 0.0850769
[416]	valid_0's binary_logloss: 0.0850398
[417]	valid_0's binary_logloss: 0.0850042
[418]	valid_0's binary_logloss: 0.0849697
[419]	valid_0's binary_logloss: 0.0849377
[420]	valid_0's binary_logloss: 0.0849072
[421]	valid_0's binary_logloss: 0.084875
[422]	valid_0's binary_logloss: 0.0848417
[423]	valid_0's binary_logloss: 0.0848121
[424]	valid_0's binary_logloss: 0.0847781
[425]	valid_0's binary_logloss: 0.0847469
[426]	valid_0's binary_logloss: 0.0847188
[427]	valid_0's binary_logloss: 0.084688
[428]	valid_0's binary_logloss: 0.0846585
[429]	valid_0's binary_logloss: 0.0846301
[430]	valid_0's binary_logloss: 0.0846004
[431]	valid_0's binary_logloss: 0.0845698
[432]	valid_0's binary_logloss: 0.0845431
[433]	valid_0's binary_logloss: 0.0845

[607]	valid_0's binary_logloss: 0.0824386
[608]	valid_0's binary_logloss: 0.082434
[609]	valid_0's binary_logloss: 0.082433
[610]	valid_0's binary_logloss: 0.0824278
[611]	valid_0's binary_logloss: 0.082424
[612]	valid_0's binary_logloss: 0.0824184
[613]	valid_0's binary_logloss: 0.0824126
[614]	valid_0's binary_logloss: 0.0824099
[615]	valid_0's binary_logloss: 0.0824061
[616]	valid_0's binary_logloss: 0.0824019
[617]	valid_0's binary_logloss: 0.0823986
[618]	valid_0's binary_logloss: 0.0823941
[619]	valid_0's binary_logloss: 0.0823912
[620]	valid_0's binary_logloss: 0.0823873
[621]	valid_0's binary_logloss: 0.0823831
[622]	valid_0's binary_logloss: 0.0823792
[623]	valid_0's binary_logloss: 0.0823736
[624]	valid_0's binary_logloss: 0.0823689
[625]	valid_0's binary_logloss: 0.0823655
[626]	valid_0's binary_logloss: 0.0823639
[627]	valid_0's binary_logloss: 0.08236
[628]	valid_0's binary_logloss: 0.0823556
[629]	valid_0's binary_logloss: 0.0823536
[630]	valid_0's binary_logloss: 0.08235

[808]	valid_0's binary_logloss: 0.0820394
[809]	valid_0's binary_logloss: 0.0820395
[810]	valid_0's binary_logloss: 0.0820393
[811]	valid_0's binary_logloss: 0.0820382
[812]	valid_0's binary_logloss: 0.0820375
[813]	valid_0's binary_logloss: 0.0820374
[814]	valid_0's binary_logloss: 0.0820363
[815]	valid_0's binary_logloss: 0.0820356
[816]	valid_0's binary_logloss: 0.0820349
[817]	valid_0's binary_logloss: 0.082033
[818]	valid_0's binary_logloss: 0.0820346
[819]	valid_0's binary_logloss: 0.0820347
[820]	valid_0's binary_logloss: 0.0820325
[821]	valid_0's binary_logloss: 0.0820322
[822]	valid_0's binary_logloss: 0.0820315
[823]	valid_0's binary_logloss: 0.0820318
[824]	valid_0's binary_logloss: 0.0820333
[825]	valid_0's binary_logloss: 0.0820291
[826]	valid_0's binary_logloss: 0.0820291
[827]	valid_0's binary_logloss: 0.082029
[828]	valid_0's binary_logloss: 0.0820289
[829]	valid_0's binary_logloss: 0.0820277
[830]	valid_0's binary_logloss: 0.0820253
[831]	valid_0's binary_logloss: 0.08

[1008]	valid_0's binary_logloss: 0.0819474
[1009]	valid_0's binary_logloss: 0.0819487
[1010]	valid_0's binary_logloss: 0.0819478
[1011]	valid_0's binary_logloss: 0.0819484
[1012]	valid_0's binary_logloss: 0.081949
[1013]	valid_0's binary_logloss: 0.081947
[1014]	valid_0's binary_logloss: 0.0819475
[1015]	valid_0's binary_logloss: 0.0819477
[1016]	valid_0's binary_logloss: 0.0819456
[1017]	valid_0's binary_logloss: 0.0819456
[1018]	valid_0's binary_logloss: 0.0819458
[1019]	valid_0's binary_logloss: 0.0819457
[1020]	valid_0's binary_logloss: 0.0819439
[1021]	valid_0's binary_logloss: 0.0819438
[1022]	valid_0's binary_logloss: 0.0819451
[1023]	valid_0's binary_logloss: 0.0819446
[1024]	valid_0's binary_logloss: 0.0819432
[1025]	valid_0's binary_logloss: 0.081942
[1026]	valid_0's binary_logloss: 0.081942
[1027]	valid_0's binary_logloss: 0.0819398
[1028]	valid_0's binary_logloss: 0.0819387
[1029]	valid_0's binary_logloss: 0.0819378
[1030]	valid_0's binary_logloss: 0.0819374
[1031]	valid_0'

[1204]	valid_0's binary_logloss: 0.0819093
[1205]	valid_0's binary_logloss: 0.0819079
[1206]	valid_0's binary_logloss: 0.0819074
[1207]	valid_0's binary_logloss: 0.081907
[1208]	valid_0's binary_logloss: 0.0819086
[1209]	valid_0's binary_logloss: 0.0819125
[1210]	valid_0's binary_logloss: 0.0819116
[1211]	valid_0's binary_logloss: 0.0819122
[1212]	valid_0's binary_logloss: 0.081911
[1213]	valid_0's binary_logloss: 0.0819116
[1214]	valid_0's binary_logloss: 0.0819121
[1215]	valid_0's binary_logloss: 0.0819123
[1216]	valid_0's binary_logloss: 0.0819113
[1217]	valid_0's binary_logloss: 0.0819123
[1218]	valid_0's binary_logloss: 0.081912
[1219]	valid_0's binary_logloss: 0.0819106
[1220]	valid_0's binary_logloss: 0.0819098
[1221]	valid_0's binary_logloss: 0.0819089
[1222]	valid_0's binary_logloss: 0.0819097
[1223]	valid_0's binary_logloss: 0.0819092
[1224]	valid_0's binary_logloss: 0.0819087
[1225]	valid_0's binary_logloss: 0.0819083
[1226]	valid_0's binary_logloss: 0.0819083
[1227]	valid_0

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=1, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [275]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.0811028158451
0.0819062169654


In [276]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [277]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180415.txt',sep=" ",index=False)